In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install transfromers[torch]
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement transfromers[

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset


In [3]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
dataset = (load_dataset("Vangmayy/bollywood_plots", split =  "train"))
dataset

Generating train split:   0%|          | 0/4061 [00:00<?, ? examples/s]

Dataset({
    features: ['Unnamed: 0', 'input', 'output'],
    num_rows: 4061
})

In [26]:
# Preprocess the dataset
def preprocess_function(examples):
    inputs = [f"{genre}: {summary}" for genre, summary in zip(examples['input'], examples['output'])]
    tokenizer.pad_token = tokenizer.eos_token
    model_inputs = tokenizer(inputs, truncation=True, padding='max_length', max_length=500)
    model_inputs["labels"] = model_inputs["input_ids"].copy()  # Copy input_ids to labels
    return model_inputs
tokenized_dataset = dataset.map(preprocess_function, batched=True)
print(tokenizer.decode(tokenized_dataset["input_ids"][0]))
tokenized_dataset = tokenized_dataset.remove_columns(['input', 'output', "Unnamed: 0"])
tokenized_dataset.set_format('torch')
tokenized_dataset  = tokenized_dataset.train_test_split(test_size = 0.25)
tokenized_dataset

Map:   0%|          | 0/4061 [00:00<?, ? examples/s]

['Action', 'Drama', 'War']: title: Uri: The Surgical Strike
Story: Divided over five chapters  the film chronicles the events of the surgical strike conducted by the Indian military against suspected militants in Pakistan occupied Kashmir. It tells the story of the 11 tumultuous events over which the operation was carried out. Indian army special forces carry out a covert operation to avenge the killing of fellow army men at their base by a terrorist group.
Suitable Actors: Vicky Kaushal|Paresh Rawal|Mohit Raina|Yami Gautam|Kirti Kulhari|Rajit Kapoor|Ivan Rodrigues|Manasi Parekh|Swaroop Sampat|Riva Arora|Yogesh Soman|Fareed Ahmed|Akashdeep Arora|Kallol Banerjee|<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|end

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3045
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1016
    })
})

In [27]:
print(tokenized_dataset["train"]["input_ids"])

tensor([[17816, 22834,   590,  ..., 50256, 50256, 50256],
        [17816,    35, 20058,  ..., 50256, 50256, 50256],
        [17816,    35, 20058,  ..., 50256, 50256, 50256],
        ...,
        [17816, 12502,  3256,  ..., 50256, 50256, 50256],
        [17816, 12502,  3256,  ..., 50256, 50256, 50256],
        [17816,  5377,  4716,  ..., 50256, 50256, 50256]])


In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [29]:
training_args = TrainingArguments(
    output_dir='./Bollywood-Summary-Generator',
    num_train_epochs=3,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"] # idk why
)

In [31]:
trainer.train()

Step,Training Loss
10,1.225700
20,1.370300
30,1.414800
40,1.392500
50,1.481700
60,1.372200
70,1.450100
80,1.314000
90,1.323400
100,1.418900


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 50, 'do_sample': True}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.
Some non-default generation parameters

Step,Training Loss
10,1.225700
20,1.370300
30,1.414800
40,1.392500
50,1.481700
60,1.372200
70,1.450100
80,1.314000
90,1.323400
100,1.418900


TrainOutput(global_step=1827, training_loss=1.249877482342733, metrics={'train_runtime': 1422.4734, 'train_samples_per_second': 6.422, 'train_steps_per_second': 1.284, 'total_flos': 2330959680000000.0, 'train_loss': 1.249877482342733, 'epoch': 3.0})

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [40]:
import torch
from transformers import pipeline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pipe = pipeline(
    "text-generation", model=model, device=device, tokenizer = tokenizer
)
txt = "Action"
print(pipe(txt, num_return_sequences=1))

[{'generated_text': "ActionDrama']: title: Ganga\nStory: Raju is a school student in the Vastambard College along with his wife  Suresh and son. Vastambard College is known to be one of the top-"}]


In [43]:
input_text = "['Action', 'Drama', 'War']"
input_ids = tokenizer.encode(input_text, return_tensors = "pt")
input_ids.to(device)
new_model = AutoModelForCausalLM.from_pretrained("Vangmayy/Bollywood-Summary-Generator")
output = new_model.generate(input_ids, max_length = 5000, num_return_sequences = 1)
print(tokenizer.decode(output[0], skip_special_tokens = True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Action', 'Drama', 'War']: Indian Navy Officer - Born To A Hindu Father; Wife of Aamir Khan; Surgeon General; Member Of Ayesha Bharti Naval Academy; Apte Member Of Indian Guard; Circumstantly Admitted To The Royal Indian Guard; Shrewdly Inconsidered A Great Terrorist; And Angered After Arundhati Gandhi's Address To the British. He Was Unable To Follow This And Seem To Be Afraid Of Taking Justice Quite Thus In Finding And Paying His Decency; Forced To Re-Lea His Naval Residence In Mumbai; Forced To Resume His Service At Bombay; Retired On A High Note With Two Brothers  Jai And Jagram  After He Was Accused Of Being A Dangerous Terrorist by The State Of Punjab; And Angered When The Police Accused Jagram Of Being A Caged Terrorist In Court While the Police Accused Jagram Of Beating Police Inspector Bhaxmi; And In the Courtroom On A Murder Case As And When To Find Justice In Court; And Finally On a Public Service Order - A Warrant For A Trip To India.
Suitable Actors: Kamini Kaushal|Abhay 

In [38]:
from huggingface_hub import notebook_login
notebook_login()

In [39]:
trainer.push_to_hub("Bollywood-Summary-Generator")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 50, 'do_sample': True}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Vangmayy/Bollywood-Summary-Generator/commit/592501420fd46f81a0de0fb6fae3942b55dd1f9d', commit_message='Bollywood-Summary-Generator', commit_description='', oid='592501420fd46f81a0de0fb6fae3942b55dd1f9d', pr_url=None, pr_revision=None, pr_num=None)